## Поток выполнения
### Модальные диалоги с `@st.dialog`

**Краткий обзор**  
> Начиная с версии **Streamlit 1.42.0**, появилась возможность создавать модальные диалоги с помощью декоратора `@st.dialog`. Это удобный способ отобразить дополнительные поля ввода и информацию во всплывающем окне без перезапуска всего приложения.
#### Основная концепция
- **Декоратор `@st.dialog`**  
    Позволяет объявить функцию, при вызове которой появляется модальное окно. Все команды вывода внутри этой функции отрисовываются внутри диалога.
- **Закрытие диалога**  
    Пользователь может закрыть диалог, кликнув вне диалогового окна, на иконку “X” или нажимая `ESC`. Для программного закрытия нужно вызвать `st.rerun()` внутри диалога.
- **Ограничение:**  
    В рамках одного прогона кода в Streamlit **можно открыть только один диалог**.
- **Ререндер**  
    При взаимодействии с виджетами (кнопками, вводом текста) внутри диалога, перезапускается **только диалог-функция**, а не весь скрипт.
- **Взаимодействие с Session State**  
    Результаты взаимодействия (например, введённый текст) можно сохранять в `st.session_state`, чтобы использовать их вне диалога.
- **Ширина диалога** (`width`)  
    По умолчанию используется `"small"` (500 пикселей). Можно указать `"large"`, чтобы сделать диалог шириной ~750 пикселей.

**Сторонний вывод**  
> Вывод в боковую панель (`st.sidebar`) внутри диалога **не поддерживается**.
#### Пример кода
```python
import streamlit as st

@st.dialog("Оставьте голос")  # Задаём заголовок диалога
def vote(item):
    st.write(f"Почему вы выбрали {item}?")
    reason = st.text_input("Поясните выбор")
    if st.button("Отправить"):
        st.session_state.vote = {"item": item, "reason": reason}
        st.rerun()  # Закрывает диалог и перезапускает приложение

if "vote" not in st.session_state:
    st.write("Голосуйте за лучший вариант")
    if st.button("A"):
        vote("A")
    if st.button("B"):
        vote("B")
else:
    st.write(
        f"Вы проголосовали за {st.session_state.vote['item']} "
        f"по причине: {st.session_state.vote['reason']}"
    )
```
**Основные моменты**
> 
> 1. Декоратор `@st.dialog` делает из функции модальное окно.
> 2. Диалог можно закрыть кликом, ESC или вызвав `st.rerun()`.
> 3. При вводе данных внутри диалога перезапускается только диалог-функция.
> 4. Данные из диалога сохраняются в Session State для общего использования.
### Использование `st.form`

**Краткий обзор**  
> `st.form` позволяет группировать виджеты в **форму** с кнопкой "Submit", отправляя их значения **одним пакетом**. Это полезно для обработки нескольких входных данных одновременно.
#### Основные особенности `st.form`
✅ **Объединяет виджеты** и отправляет их данные разом.  
✅ **Обязательная кнопка** `st.form_submit_button`.  
✅ **Поддерживает `with`-нотацию** для удобного создания.  
✅ **Работает в `st.sidebar`, колонках и др.**  
✅ **Возможность отправки при нажатии Enter** (`enter_to_submit=True`).  
❌ **Не поддерживает `st.button` и `st.download_button` внутри формы**.  
❌ **Формы нельзя вкладывать друг в друга**.  
❌ **Только `st.form_submit_button` поддерживает `callback`**.
#### Параметры `st.form`

|Параметр|Описание|
|---|---|
|`key`|Уникальный идентификатор формы (обязателен).|
|`clear_on_submit`|Сбрасывать ли данные после отправки (`False` по умолчанию).|
|`enter_to_submit`|Позволяет ли Enter отправлять форму (`True` по умолчанию).|
|`border`|Показывать ли границу формы (`True` по умолчанию).|
#### Примеры кода
**Использование `with`-нотации (предпочтительный метод)**
```python
import streamlit as st

with st.form("my_form"):
    st.write("Форма ввода данных")
    slider_val = st.slider("Выберите значение")
    checkbox_val = st.checkbox("Подтвердите")

    submitted = st.form_submit_button("Отправить")
    if submitted:
        st.write("Значения:", slider_val, checkbox_val)

st.write("Код после формы")
```
**Создание формы без `with` (альтернативный метод)**
```python
import streamlit as st

form = st.form("my_form")
form.slider("Ползунок в форме")
st.slider("Ползунок вне формы")  # Этот элемент не входит в форму!

form.form_submit_button("Отправить")
```
**Очистка формы после отправки**
```python
import streamlit as st

with st.form("reset_form", clear_on_submit=True):
    name = st.text_input("Введите имя")
    age = st.number_input("Возраст", min_value=0, max_value=100)
    
    submitted = st.form_submit_button("Отправить")
    if submitted:
        st.write(f"Имя: {name}, Возраст: {age}")
```
**Основные моменты**
> 
> - `st.form` группирует элементы и отправляет их данные одновременно.
> - Всегда использовать `st.form_submit_button` для обработки формы.
> - Можно автоматически отправлять форму нажатием **Enter** (`enter_to_submit=True`).
> - Для сброса значений после отправки установите `clear_on_submit=True`.
> - **Не использовать `st.button` внутри формы** — он не будет работать.
### Использование `@st.fragment`
**Краткий обзор**  
> Декоратор `@st.fragment` позволяет запускать функцию **независимо** от основной части приложения, **без его полного перезапуска**. Это полезно для обновления интерфейса **без перерисовки всего приложения**.
#### Основные особенности `@st.fragment`
✅ Позволяет перезапускать только фрагмент, а не всё приложение.  
✅ Можно задать интервал автоматического обновления (`run_every`).  
✅ Доступ к `st.session_state` для передачи данных между фрагментами и приложением.  
✅ Может обновлять только свой **главный блок**, а не внешние контейнеры.  
❌ **Не поддерживает `st.sidebar`** — если нужно, вызывайте его внутри `with st.sidebar`.  
❌ **Нельзя рендерить виджеты во внешние контейнеры**.
#### Параметры `@st.fragment`

|Параметр|Описание|
|---|---|
|`func`|Функция, которая станет фрагментом.|
|`run_every`|Автоматический интервал обновления (в секундах, `timedelta`, либо `"1h30m"`).|

#### 🛠 Примеры использования
📌 **Базовый пример: быстрый ререндер фрагмента**
```python
import streamlit as st
import time

@st.fragment
def release_the_balloons():
    st.button("Запустить воздушные шары", help="Перезапуск фрагмента")
    st.balloons()  # Вызов анимации

with st.spinner("Надуваем шары..."):
    time.sleep(5)  # Симуляция задержки

release_the_balloons()
st.button("Надуть ещё шаров", help="Полный перезапуск")
```

🔹 **Как это работает?**
- Основной код (`time.sleep(5)`) выполняется **один раз**.
- Кнопка `"Запустить воздушные шары"` вызывает `release_the_balloons()`, **но не перезапускает всю страницу**.
- Кнопка `"Надуть ещё шаров"` уже **перезапускает всю страницу**.
📌 **Разница между ререндером фрагмента и приложения**
```python
import streamlit as st

if "app_runs" not in st.session_state:
    st.session_state.app_runs = 0
    st.session_state.fragment_runs = 0

@st.fragment
def my_fragment():
    st.session_state.fragment_runs += 1
    st.button("Перезапустить только фрагмент")
    st.write(f"Фрагмент запускался {st.session_state.fragment_runs} раз.")

st.session_state.app_runs += 1
my_fragment()

st.button("Перезапустить всё приложение")
st.write(f"Приложение запускалось {st.session_state.app_runs} раз.")
st.write(f"Приложение видит, что фрагмент запускался {st.session_state.fragment_runs} раз.")
```

🔹 **Как это работает?**
- `"Перезапустить только фрагмент"` обновляет только блок `my_fragment()`.
- `"Перезапустить всё приложение"` сбрасывает и счётчик фрагмента, и основной счётчик.

📌 **Программное обновление фрагмента**
```python
import streamlit as st

if "clicks" not in st.session_state:
    st.session_state.clicks = 0

@st.fragment
def count_to_five():
    if st.button("Добавить +1"):
        st.session_state.clicks += 1
        if st.session_state.clicks % 5 == 0:
            st.rerun()  # Перезапуск всего приложения после каждого 5-го клика

count_to_five()
st.header(f"Количество кликов: {st.session_state.clicks // 5 * 5}")

if st.button("Проверить счётчик"):
    st.toast(f"## Всего кликов: {st.session_state.clicks}")
```

🔹 **Как это работает?**
- Кнопка `"Добавить +1"` увеличивает счётчик.
- Каждые **5 кликов** вызывается `st.rerun()`, перезапуская всё приложение.
- `"Проверить счётчик"` показывает общее количество кликов через `st.toast()`.
**Основные моменты**
> 
> - `@st.fragment` позволяет перезапускать только фрагмент, а не всю страницу.
> - Можно задать автообновление через `run_every`.
> - Для взаимодействия с приложением использовать `st.session_state`.
> - **Будьте осторожны**: `st.fragment` не очищает внешние контейнеры при обновлении.
### Использование `st.rerun`
**Краткий обзор**  
> `st.rerun()` мгновенно **перезапускает скрипт** Streamlit, останавливая выполнение текущего кода. Это полезно для обновления данных, динамического изменения интерфейса и обработки событий.
#### Основные особенности `st.rerun`
✅ Позволяет **перезапустить всё приложение** или только фрагмент (`@st.fragment`).  
✅ Сбрасывает выполнение текущего кода **и запускает его заново**.  
✅ Можно использовать для мгновенной реакции на события (`on_click`).  
❌ **Частые перезапуски могут замедлить работу** приложения.  
❌ **Может вызвать бесконечный цикл**, если неправильно использовать.  
❌ **Лучше заменить на callbacks (`on_click`) или контейнеры**, где это возможно.
#### Параметры `st.rerun`

| Параметр           | Описание                                                                         |
| ------------------ | -------------------------------------------------------------------------------- |
| `scope="app"`      | Перезапускает всё приложение (**по умолчанию**).                                 |
| `scope="fragment"` | Перезапускает **только фрагмент**, но работает **только внутри `@st.fragment`**. |
**Ограничение**
> 
> - **`st.rerun(scope="fragment")` работает только в `@st.fragment`**.
> - Если `st.rerun(scope="fragment")` вызывается во время **полного перезапуска** приложения — произойдет ошибка `StreamlitAPIException`.
#### 🛠 Примеры использования

📌 **Простой перезапуск приложения**
```python
import streamlit as st

if "value" not in st.session_state:
    st.session_state.value = "Заголовок"

st.header(st.session_state.value)

if st.button("Изменить заголовок"):
    st.session_state.value = "Обновленный заголовок"
    st.rerun()  # Перезапуск всего скрипта
```
🔹 **Как это работает?**
- После нажатия кнопки `"Изменить заголовок"`, переменная в `st.session_state` обновляется.
- `st.rerun()` **немедленно перезапускает** код и отображает новый заголовок.

📌 **Перезапуск только фрагмента**
```python
import streamlit as st

if "clicks" not in st.session_state:
    st.session_state.clicks = 0

@st.fragment
def fragment_counter():
    if st.button("Добавить +1 (Фрагмент)"):
        st.session_state.clicks += 1
        st.rerun(scope="fragment")  # Перезапускает только этот блок!

st.write(f"Всего кликов: {st.session_state.clicks}")
fragment_counter()
```
🔹 **Как это работает?**
- `"Добавить +1 (Фрагмент)"` увеличивает `st.session_state.clicks`.
- `st.rerun(scope="fragment")` обновляет **только фрагмент**, а не всё приложение.

📌 **Альтернативы `st.rerun()`**
1️⃣ **Использование `on_click` (предпочтительно, если нет сложной логики)**
```python
import streamlit as st

if "value" not in st.session_state:
    st.session_state.value = "Начальный заголовок"

def update_title():
    st.session_state.value = "Обновленный заголовок"

st.header(st.session_state.value)
st.button("Обновить заголовок", on_click=update_title)  # Без перезапуска скрипта!
```
✅ **Работает без перезапуска**  
✅ **Проще и быстрее, чем `st.rerun()`**

2️⃣ **Использование контейнера (`st.container()`)**
```python
import streamlit as st

if "title" not in st.session_state:
    st.session_state.title = "Заголовок"

container = st.container()
container.header(st.session_state.title)

if st.button("Изменить заголовок"):
    st.session_state.title = "Обновленный заголовок"
```
✅ **Обновляет данные без перезапуска**  
✅ **Работает плавно и быстрее**

**Основные моменты**
> 
> - `st.rerun()` **перезапускает всё приложение** или **только фрагмент** (`scope="fragment"`).
> - Чрезмерное использование **может замедлить работу** или **создать бесконечный цикл**.
> - В большинстве случаев **лучше использовать `on_click` или `st.container()`** для обновления интерфейса **без перезапуска всего кода**.
### st.stop
```python
import streamlit as st

name = st.text_input("Name")
if not name:
  st.warning('Please input a name.')
  st.stop()
st.success("Thank you for inputting a name.")
```